## Step 1

Accessed real Lamin data via lamindb

Pulled a clean, merged AnnData object

Inspected and visualized key covariates

Are ready to move to model training



Connect to LaminDB from AWS.

Fetch a subsample of Tahoe-100M (or similar data).

Explore basic statistics.

Train a multinomial logistic regression model using sklearn.

Evaluate prediction performance.

Extract and visualize gene weights.

In [8]:
# !lamin login mikaela.koutrouli@cpr.ku.dk --key 01emvZ7k-LMC2cjF726XY31kSmKpRul8nlDCwhkW

In [ ]:
import lamindb as ln
ln.connect("laminlabs/arrayloader-benchmarks")

In [14]:
import scanpy as sc

artifact_tahoe_store = ln.Artifact.get("TuhkPw0wkzlUXN5k0000")
artifact_tahoe_store

Artifact(uid='TuhkPw0wkzlUXN5k0000', is_latest=True, key='tahoe100M/shuffled_plate3_subset_1000_100_AB', suffix='', size=3989166, hash='Y-pMgFWbhcdvZsvmXEt--w', n_files=108, space_id=1, storage_id=3, run_id=104, created_by_id=2, created_at=2025-05-10 15:05:47 UTC)

In [ ]:
import lamindb as ln
import scanpy as sc

ln.connect("laminlabs/arrayloader-benchmarks")

artifact = ln.Artifact.get(key="tahoe100M/plate3_subset_1000_100_A.h5ad")

# Force the file to be downloaded first
artifact.download()  # This should take a few seconds and print progress

# Then load into memory
adata = sc.read_h5ad(artifact.path())
print(adata)


→ connected lamindb: laminlabs/arrayloader-benchmarks
→ connected lamindb: laminlabs/arrayloader-benchmarks


In [1]:
# ln.Artifact.get("TuhkPw0wkzlUXN5k0000")

In [1]:
import lamindb as ln
# ln.connect("laminlabs/arc-virtual-cell-atlas") #laminlabs/arrayloader-benchmarks
ln.connect("laminlabs/arrayloader-benchmarks") #laminlabs/arrayloader-benchmarks


→ connected lamindb: laminlabs/arc-virtual-cell-atlas
→ connected lamindb: laminlabs/arc-virtual-cell-atlas


In [3]:
ln.Project.df()[['name', 'uid']]


,name,uid
id,,
2,scBaseCount,vdK00t9DGwHP
1,Tahoe-100M,H5MwZwyA62rG


In [ ]:
collection = ln.Collection.filter(key="tahoe100M").one()
artifacts = collection.artifacts.df()
print(artifacts[["key", "n_observations", "size"]].head())


In [24]:
# artifact = ln.Artifact.get(key="tahoe100M/plate3_subset_1000_100_A.h5ad")
# adata = sc.read_h5ad(artifact.path())  


In [18]:
# import lamindb as ln
# Get the Tahoe-100M project by name
project_tahoe = ln.Project.get(name="Tahoe-100M")
print(project_tahoe)


Project(uid='H5MwZwyA62rG', name='Tahoe-100M', is_type=False, url='https://arcinstitute.org/tools/virtualcellatlas', space_id=1, created_by_id=4, run_id=85, created_at=2025-02-26 16:03:40 UTC)


In [20]:
# project = ln.Project.get(name="Tahoe-100M")
# print(project)
# collection = ln.Collection.get(key="tahoe100")
# print(collection, "contains", collection.artifacts.count(), "artifacts")

# artifacts_df = collection.artifacts.distinct().df()
# print(artifacts_df[['key','n_observations','size']].head())


In [21]:
# artifact = ln.Artifact.get(key="plvz5n0YX1fVWbEp0000")
# adata = artifact.open()   # returns an AnnData-like object

artifact_key = artifacts_df.iloc[0]['key']
print(artifact_key)
# artifact = ln.Artifact.get(key=artifact_key)
# artifact
# adata = artifact.open()   
# adata

NameError: name 'artifacts_df' is not defined

In [10]:
# adata.obs['perturbation'].value_counts().plot(kind='bar')
# plt.title("Drug (perturbation) label distribution")
# plt.xlabel("Drug"); plt.ylabel("Count")
# plt.tight_layout()


In [ ]:
# Subset to one cell line (e.g. 'A375')
cellline = 'A375'
adata_cl = adata[adata.obs['cell_line'] == cellline, :]

# Find top 3 most common drugs within this cell line
top_drugs = adata_cl.obs['perturbation'].value_counts().nlargest(3).index.tolist()
adata_sub = adata_cl[adata_cl.obs['perturbation'].isin(top_drugs), :]

print(f"Selected {adata_sub.n_obs} cells across drugs: {adata_sub.obs['perturbation'].unique().tolist()}")


In [41]:
# 1. Connect to LaminDB instance and authenticate
# If needed, install required packages (uncomment the next line):
# !pip install lamindb anndata scanpy scikit-learn seaborn matplotlib
import os
import lamindb as ln

# # Use environment variables or insert your Lamin credentials
# user = os.getenv("mikelkou")
# api_key = os.getenv("8XF7LaZnEbIbcxvddzLTBqr74CygvH7WP7WdDMxY")
# ln.login(user=user, api_key=api_key)
# ln.connect("laminlabs/arrayloader-benchmarks")
project = ln.Project.get(name="Tahoe-100M")
print(project)
collection = ln.Collection.get(key="tahoe100")
print(collection, "contains", collection.artifacts.count(), "artifacts")


Project(uid='H5MwZwyA62rG', name='Tahoe-100M', is_type=False, url='https://arcinstitute.org/tools/virtualcellatlas', space_id=1, created_by_id=1, created_at=2025-02-26 16:03:40 UTC)
Collection(uid='BpavRL4ntRTzWEE50000', version='2025-02-25', is_latest=True, key='tahoe100', hash='GCLk4ZgQxgWspjmEUk3gIg', space_id=1, created_by_id=1, run_id=3, created_at=2025-02-26 13:51:22 UTC) contains 14 artifacts


In [42]:
artifact = ln.Artifact.filter(name="plvz5n0YX1fVWbEp0000").one()
artifact

InvalidArgument: You can query either by available fields: cell_lines, cell_markers, cell_types, collections, created_at, created_by, description, developmental_stages, diseases, ethnicities, experimental_factors, feature_sets, genes, hash, id, input_of_runs, is_latest, key, kind, n_files, n_observations, organisms, otype, pathways, phenotypes, projects, proteins, references, run, schema, size, space, storage, suffix, tissues, transform, uid, ulabels, updated_at, version, visibility
Or fix invalid feature names: name

In [16]:
ln.Artifact.filter().df()


,uid,key,description,suffix,kind,otype,size,hash,n_files,n_observations,_hash_type,_key_is_virtual,_overwrite_versions,space_id,storage_id,schema_id,version,is_latest,run_id,created_at,created_by_id,_aux,_branch_code
id,,,,,,,,,,,,,,,,,,,,,,,
444,aJIqo7bNyJAs9z0r0000,2025-02-25/h5ad/plate1_filt_Vevo_Tahoe100M_WSe...,None,.h5ad,dataset,AnnData,19070623904,9iCNcouMqfNS3HA/2GUWOA,NaN,5481420.0,md5,False,False,1,5,5.0,None,True,85.0,2025-02-25 23:22:19.737995+00:00,4,None,1
445,ZFeVfd0ugAHeWCxm0000,2025-02-25/h5ad/plate2_filt_Vevo_Tahoe100M_WSe...,None,.h5ad,dataset,AnnData,29037152127,usxviuqGbuw0RYnECCVCWw,NaN,8064658.0,md5,False,False,1,5,5.0,None,True,85.0,2025-02-25 23:22:20.113956+00:00,4,None,1
299,EsjG7XriCmfHPQQPY8sI,None,Dissection: Amygdaloid complex (AMY) - basolat...,.h5ad,None,AnnData,199778278,uut8UJWikjFM98frUFicE6,NaN,NaN,sha1-fl,True,False,1,1,NaN,None,True,16.0,2024-01-25 14:41:23.286676+00:00,1,None,1
176,XV2EdShHysGYrWQKPYmm,None,Dissection: Spinal cord - SpC subset of 20k vars,.h5ad,None,AnnData,79701636,_lN0urEG2EMYzAHf9Kg3Mc,NaN,NaN,sha1-fl,True,False,1,1,NaN,None,True,16.0,2024-01-25 12:09:48.928637+00:00,1,None,1
309,UU0rSmjkKwotOzk3KNX7,None,Dissection: Hypothalamus (HTH) - preoptic regi...,.h5ad,None,AnnData,81225345,rifwtickznxtRNoeJ8wi7L,NaN,NaN,sha1-fl,True,False,1,1,NaN,None,True,16.0,2024-01-25 14:48:10.303480+00:00,1,None,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130,nPgLBocitaiXVxcAUULS,cell-census/2023-12-15/h5ads/cfa755c1-48c5-433...,Supercluster: Choroid plexus,.h5ad,None,AnnData,85270733,NCVJvNb53mrftmFrrW2szQ,NaN,7689.0,md5-n,False,False,1,2,NaN,None,True,NaN,2024-01-16 13:21:26.342730+00:00,1,None,1
23,U2xftdsrYVKVEQ4KXlMA,cell-census/2023-12-15/h5ads/04a23820-ffa8-4be...,Supercluster: Upper rhombic lip,.h5ad,None,AnnData,710633334,JFg7Wq9Fm3zmYaPz5hd-6A,NaN,137162.0,md5-n,False,False,1,2,NaN,None,True,NaN,2024-01-16 13:09:13.293118+00:00,1,None,1
185,dFacxzR3sAoU6bGGq4xF,None,Supercluster: Midbrain-derived inhibitory subs...,.h5ad,None,AnnData,708360028,0M3biAukM9d8kZDEW2edxN,NaN,NaN,sha1-fl,True,False,1,1,NaN,None,True,16.0,2024-01-25 12:15:33.786004+00:00,1,None,1


In [ ]:
import lamindb as ln

# Step 1: Load the shared instance first (this sets it up locally)
ln.setup.load("laminlabs/arrayloader-benchmarks")


In [ ]:
import lamindb as ln

ln.connect("laminlabs/arrayloader-benchmarks")
collection = ln.Collection.filter(key="gather").one()
adata = collection.load(join="inner")


In [1]:
# Cell 1: Imports
import lamindb as ln
import scanpy as sc
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(style="whitegrid")


In [6]:
# Cell 2: Connect to the Lamin instance
# ln.connect("laminlabs/arrayloader-benchmarks")
# !lamin load laminlabs/arrayloader-benchmarks


In [2]:
# Cell 3: Load the data collection
collection = ln.Collection.filter(key="gather").one()

# Ensure all .h5ad parts are downloaded (if not already staged)
for artifact in collection.artifacts:
    artifact.stage()


InstanceNotSetupError: To use lamindb, you need to connect to an instance.

Connect to an instance: `ln.connect()`. Init an instance: `ln.setup.init()`.

If you used the CLI to set up lamindb in a notebook, restart the Python session.


In [ ]:
# Cell 4: Load the joined dataset (inner join = common genes)
adata = collection.load(join="inner")
adata

In [ ]:
# Cell 5: Check available metadata (obs)
print("Cells:", adata.n_obs)
print("Genes:", adata.n_vars)
print("Metadata columns:", adata.obs.columns.tolist())


In [ ]:
# Cell 6: Visualize covariate distributions
adata.obs["cell_type"].value_counts().plot.bar(
    figsize=(8, 4), title="Cell type distribution"
)
plt.ylabel("Cell count")
plt.tight_layout()
plt.show()


In [ ]:
adata.obs["batch"].value_counts().plot.bar(
    figsize=(6, 3), title="Batch distribution"
)
plt.ylabel("Cell count")
plt.tight_layout()
plt.show()
